In [38]:
# !pip install evaluate

# Import Tools

In [39]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

train_data = pd.read_csv(r"D:/Data science Notes/Python/Projects/Text summarization/dataset/samsum-train.csv")
validation_data = pd.read_csv(r"D:/Data science Notes/Python/Projects/Text summarization/dataset/samsum-validation.csv")

# Load Datasets

In [40]:


# Display a sample
train_data.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [41]:
train_data = train_data.sample(n=4000,random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(n=500, random_state=42).reset_index(drop=True)

# Data Preprocessing 

In [42]:
# Clean the text by removing unwanted characters
import re

def clean_text(text):
    text = re.sub(r'\r\n', ' ', text)  # Remove carriage returns and line breaks
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'<.*?>', '', text)  # Remove any XML tags
    text = text.strip().lower()  # Strip and convert to lower case
    return text

# Apply cleaning to dialogue and summary columns
train_data['dialogue'] = train_data['dialogue'].apply(clean_text)
train_data['summary'] = train_data['summary'].apply(clean_text)

validation_data['dialogue'] = validation_data['dialogue'].apply(clean_text)
validation_data['summary'] = validation_data['summary'].apply(clean_text)


# Display a sample after cleaning
train_data

,id,dialogue,summary
0,13811908,violet: hi! i came across this austin's articl...,violet sent claire austin's article.
1,13716431,pat: so does anyone know when the stream is go...,pat and lou are waiting for the stream but kev...
2,13810214,jane: jane: whaddya think? shona: this ur tin...,jane is updating her tinder profile tonight an...
3,13729823,"adam: do u have a map of paris? tom: yes, why?...",tom has a map of paris.
4,13681400,"frank: hi, how's the family? mike: great! sam'...","mike is happy, because sam's moved out. mike a..."
...,...,...,...
3995,13681041,barry: hello buddy michael: hey barry: do you ...,barry and michael will watch football instead ...
3996,13818705,karen: hey lisa. larissa and me have recently ...,karen and larissa moved to belgium and ask lis...
3997,13821859,"miles: hey, guys, i'm so sorry, but i missed t...","miles has missed the bus, so he may be 15 minu..."
3998,13812716,emma: did you finish the book i gave you? liam...,"emma gave ""the first fifteen lives of harry au..."


# Tokenization

In [43]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
input_max_len = max(len(tokenizer.encode(text)) for text in train_data['dialogue'])
output_max_len = max(len(tokenizer.encode(text)) for text in train_data['summary'])

input_max_len, output_max_len

Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


(1224, 103)

In [44]:
# Preprocessing function for tokenization
def preprocess_function(examples):
    # Tokenize the dialogue and summary
    inputs = tokenizer(examples["dialogue"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples["summary"], padding="max_length", truncation=True, max_length=150)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply the preprocessing
train_dataset = train_data.apply(preprocess_function, axis=1)
val_dataset = validation_data.apply(preprocess_function, axis=1)

In [45]:
train_dataset[0]

{'input_ids': [25208, 10, 7102, 55, 3, 23, 764, 640, 48, 403, 17, 77, 31, 7, 1108, 11, 3, 23, 816, 24, 25, 429, 253, 34, 1477, 25208, 10, 3, 7997, 15, 10, 7102, 55, 3, 10, 61, 2049, 6, 68, 3, 23, 31, 162, 641, 608, 34, 5, 3, 10, 61, 3, 7997, 15, 10, 68, 2049, 21, 1631, 81, 140, 3, 10, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Fine Tuning Model

In [46]:
# Model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define training arguments
training_args = TrainingArguments(
    output_dir="D:/Data science Notes/Python/Projects/Text summarization/results",          # output directory for checkpoints
    num_train_epochs=6,              # number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="D:/Data science Notes/Python/Projects/Text summarization/logs",            # directory for storing logs
    logging_steps=50,                # how often to log training info
    save_steps=500,                  # how often to save a model checkpoint
    eval_steps=50,                   # how often to run evaluation
    eval_strategy="epoch",     # Ensure evaluation happens every `epoch`
)

# Trainer setup
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

  0%|          | 2/3000 [03:00<75:09:40, 90.25s/it]
                                                   
  2%|▏         | 50/3000 [03:54<3:51:11,  4.70s/it]

{'loss': 14.5107, 'grad_norm': 80.48126983642578, 'learning_rate': 5e-06, 'epoch': 0.1}


                                                    
  3%|▎         | 100/3000 [08:02<4:02:36,  5.02s/it]

{'loss': 11.3336, 'grad_norm': 71.20511627197266, 'learning_rate': 1e-05, 'epoch': 0.2}


                                                    
  5%|▌         | 150/3000 [12:11<3:54:24,  4.93s/it]

...

                                                     
100%|██████████| 3000/3000 [5:02:56<00:00,  5.94s/it]

{'loss': 0.3533, 'grad_norm': 0.5343660712242126, 'learning_rate': 0.0, 'epoch': 6.0}

































































                                                     
                                                  

100%|██████████| 3000/3000 [5:04:33<00:00,  5.94s/it]

                                                     
100%|██████████| 3000/3000 [5:04:33<00:00,  6.09s/it]

{'eval_loss': 0.34824591875076294, 'eval_runtime': 96.1606, 'eval_samples_per_second': 5.2, 'eval_steps_per_second': 0.655, 'epoch': 6.0}
{'train_runtime': 18273.3507, 'train_samples_per_second': 1.313, 'train_steps_per_second': 0.164, 'train_loss': 0.9042984975179037, 'epoch': 6.0}


TrainOutput(global_step=3000, training_loss=0.9042984975179037, metrics={'train_runtime': 18273.3507, 'train_samples_per_second': 1.313, 'train_steps_per_second': 0.164, 'total_flos': 3248203235328000.0, 'train_loss': 0.9042984975179037, 'epoch': 6.0})

# Save and load model

In [47]:
# Save the fine-tuned model
model.save_pretrained("D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model")
tokenizer.save_pretrained("D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model")

('D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model\\tokenizer_config.json',
 'D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model\\special_tokens_map.json',
 'D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model\\spiece.model',
 'D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model\\added_tokens.json')

In [48]:
# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model")
tokenizer = T5Tokenizer.from_pretrained("D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model")

# Summarization System

In [49]:
# Ensure the model is on the correct device (GPU if available)
device = model.device  # Get the device the model is on

def summarize_dialogue(dialogue):
    dialogue = clean_text(dialogue)  # Assuming clean_text is defined
    inputs = tokenizer(dialogue, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    
    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate summary
    outputs = model.generate(
        inputs["input_ids"], 
        max_length=150,  
        num_beams=4, 
        early_stopping=True
    )
    
    # Decode the generated summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [50]:
# Test with a sample input
sample_dialogue = """
Violet: Hey Claire! I was reading an article about Austin and thought you might find it interesting! 
Violet: It's about the current trends in urban development and how cities are planning for the future.
Violet: Here, let me share the link: <file_other>
Claire: Oh wow, that sounds like an insightful read. But I've actually already read that one last week. 
Claire: It was really interesting though, especially the part about sustainable architecture in cities. 
Claire: You know, I've been following these urban planning discussions for a while now.
Violet: Oh, I didn’t know that! Well, I’ll look for something else then, maybe something about eco-friendly cities or tech innovations.
Claire: That would be awesome! Let me know if you find something cool.
Violet: Sure, I’ll keep you posted. Thanks for the feedback!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)


Summary: claire was reading an article about austin and thought she might find it interesting. violet has already read that one last week.


In [51]:
# Test with a dialogue on a different topic
sample_dialogue = """
John: Hey Sarah, have you seen the latest tech gadget reviews? I found this new smartwatch that's supposed to have amazing health tracking features.
John: It tracks heart rate, blood oxygen levels, sleep patterns, and even stress levels! It sounds like something right up your alley. 
Sarah: That sounds really interesting! But I’ve been trying to cut down on tech distractions. I’ve heard these devices can be really overwhelming sometimes.
Sarah: I do think it’s cool that they can track so many health metrics though. I’m curious how accurate they really are.
John: Yeah, me too! There are also some new smartphones coming out with even better cameras and longer battery life. The new flagship model from XYZ brand has some insane specs.
Sarah: Ooh, I haven’t kept up with phones recently, but I’ve heard the camera quality is getting ridiculously good. It’s almost like a professional camera in your pocket now!
Sarah: Still, I feel like I’m fine with my current phone for now. I don’t really feel the need to upgrade unless something really groundbreaking comes out.
John: Totally understand that. It’s the same with me. But I think the battery life improvements are enough to make me consider it. I hate running out of battery when I’m out and about.
Sarah: That’s fair! I’m always worried about battery life too. Honestly, I think phones should last at least two full days on a single charge by now.
John: I agree! It’s so annoying when your phone dies in the middle of the day. I wonder if we’ll ever get to a point where we don’t have to charge our phones every day.
Sarah: That would be amazing! I think as tech improves, battery tech might also catch up. Let’s hope the next generation of phones can last longer!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)


Summary: sarah has seen the latest tech gadget reviews. john found this smartwatch that's supposed to have amazing health tracking features. there are also new smartphones coming out with even better cameras and longer battery life.


In [52]:
# Test with a dialogue on a current news topic
sample_dialogue = """
Reporter: In today's news, the latest climate change report reveals alarming global temperature rises. According to the Intergovernmental Panel on Climate Change (IPCC), the Earth’s temperature is on track to rise by 1.5°C within the next two decades.
Reporter: This is expected to lead to more frequent and severe heatwaves, flooding, and extreme weather events. Coastal cities are at particular risk due to rising sea levels.
Expert: The report emphasizes that immediate action is needed to prevent catastrophic consequences. We need to significantly reduce carbon emissions and transition to renewable energy sources.
Expert: If global temperatures increase by more than 1.5°C, we could face irreversible damage to ecosystems, agriculture, and water supply. It will have a devastating impact on biodiversity as well.
Reporter: The IPCC also stresses the importance of individual action. Governments must set stronger policies, but individuals can help by reducing waste, conserving water, and supporting green initiatives.
Expert: It's not just about the big changes; small actions like using public transportation, reducing meat consumption, and recycling can collectively make a significant difference.
Reporter: With the next UN Climate Summit coming up next month, world leaders will need to prioritize climate action. The stakes have never been higher for our planet’s future.
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)


Summary: the latest climate change report reveals alarming global temperature rises. the earth’s temperature is on track to rise by 1.5°c within the next two decades.


In [53]:
sample_dialogue = """
Rita – Hey Tina? Is it you?

Tina – Oh Rita! How are you? It’s been a long time.

Rita – I am fine, what about you? Yes, we last met during the board exams.

Tina – I’m good too.

Rita – What are you doing now?

Tina – Well, I have started my undergraduate studies in English Honours at St. Xaviers College in Mumbai.

Rita – Wow! You finally got to study the subject you loved the most in school.

Tina – True. What about you Rita? Wasn’t History your favourite subject?

Rita – You guessed it right. I took up History Honours in Lady Shri Ram College for Women in Delhi.

Tina – That’s nice. I am so happy for you.

Rita – I am happy for you too. Let’s meet up again soon.

Tina – Yes, sure! We have a lot to catch up on.

Rita – Bye for now. I have to pick up my sister from tuition. Take care.

Tina – Bye, will see you soon.
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: tina has started her undergraduate studies in english honours at st. xaviers college in mumbai. she took up history honours in lady shri ram college for women in delhi.


# Download Mode To Your Machine

In [54]:
import shutil

# Path to the directory containing the fine-tuned model
model_dir = "D:/Data science Notes/Python/Projects/Text summarization/saved_summary_model"

# Output zip file path
output_zip_path = "saved_summary_model.zip"

# Create a zip archive
shutil.make_archive(base_name="saved_summary_model", format="zip", root_dir=model_dir)

'd:\\Data science Notes\\Python\\Projects\\Text summarization\\saved_summary_model.zip'

In [55]:
from IPython.display import FileLink

# Display a download link
FileLink(output_zip_path)

d:\Data science Notes\Python\Projects\Text summarization\saved_summary_model.zip